In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import silhouette_score
import gensim.models

In [12]:
df = pd.read_csv('data/data/usa_2018/ticker_data_preprocessed.csv', index_col=0)
print(df.shape)
df.head()

(467, 1255)


,2018-01-03 00:00:00,2018-01-04 00:00:00,2018-01-05 00:00:00,2018-01-08 00:00:00,2018-01-09 00:00:00,2018-01-10 00:00:00,2018-01-11 00:00:00,2018-01-12 00:00:00,2018-01-16 00:00:00,2018-01-17 00:00:00,...,2022-12-13 00:00:00,2022-12-14 00:00:00,2022-12-15 00:00:00,2022-12-16 00:00:00,2022-12-19 00:00:00,2022-12-20 00:00:00,2022-12-21 00:00:00,2022-12-22 00:00:00,2022-12-23 00:00:00,sector
A,0.025444,-0.007501,0.015988,0.002146,0.024554,-0.013655,0.000141,0.013136,-0.006971,0.011652,...,0.015000,-0.018077,-0.025515,-0.010341,-0.015740,0.006669,0.014602,-0.007196,0.001476,Healthcare
AAL,-0.012266,0.006305,-0.000380,-0.009877,-0.000959,0.032642,0.049089,0.036335,-0.008380,0.003105,...,-0.052113,-0.008172,-0.031461,-0.009281,-0.025761,0.004006,0.039904,-0.036071,0.011943,Industrials
AAP,0.009049,0.036899,0.010631,-0.007042,-0.008080,0.000905,0.021340,0.026472,-0.017595,0.012730,...,-0.001713,-0.015239,-0.001603,-0.021434,0.005280,-0.010788,0.020878,-0.001476,0.008446,Consumer Cyclical
AAPL,-0.000174,0.004645,0.011385,-0.003714,-0.000115,-0.000229,0.005680,0.010326,-0.005082,0.016516,...,0.006782,-0.015536,-0.046854,-0.014579,-0.015910,-0.000529,0.023809,-0.023773,-0.002798,Technology
ABBV,0.015649,-0.005703,0.017408,-0.016022,0.007538,-0.005487,-0.004213,0.010779,0.021427,0.018246,...,-0.003206,0.000425,-0.010676,-0.016064,0.006294,-0.005635,0.010151,0.006535,-0.001041,Healthcare


In [13]:
df_pct = df.drop(['sector'], axis=1).T
df_pct.index = pd.to_datetime(df_pct.index)

tickers_list = df_pct.columns.tolist()

df_pct.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
2018-01-03,0.025444,-0.012266,0.009049,-0.000174,0.015649,0.003722,0.017300,0.002211,0.004615,0.012406,...,-0.010834,-0.006693,0.019640,-0.003426,0.012193,-0.000858,0.006932,0.019863,-0.001183,0.004598
2018-01-04,-0.007501,0.006305,0.036899,0.004645,-0.005703,-0.002225,0.017516,-0.001697,0.011841,-0.001094,...,0.005415,-0.007791,0.001384,-0.000149,0.006676,0.010180,-0.001441,0.019760,0.004147,0.005964
2018-01-05,0.015988,-0.000380,0.010631,0.011385,0.017408,0.012104,0.015408,0.002890,0.008249,0.004053,...,0.006671,-0.007003,-0.000806,0.014051,-0.001874,0.005828,0.009941,0.015576,0.000393,0.011444
2018-01-08,0.002146,-0.009877,-0.007042,-0.003714,-0.016022,0.016576,0.027086,-0.002882,0.007991,0.001745,...,-0.013314,0.007480,0.004496,0.006781,0.003611,0.001690,0.001905,0.009951,-0.004914,0.011996
2018-01-09,0.024554,-0.000959,-0.008080,-0.000115,0.007538,0.006398,0.009432,0.001700,0.003335,-0.002069,...,0.006778,-0.011667,-0.004246,-0.041728,0.000288,-0.002651,-0.016083,0.030643,0.023509,0.011719


In [7]:
scores = []
X = df_pct.values.reshape(-1,1)

In [48]:
X.shape

(520260, 1)

In [49]:
model = KMeans(n_clusters=200)
model.fit(X)
labels = model.predict(X)

In [50]:
X.shape, labels.shape

((520260, 1), (520260,))

In [51]:
mapping = {x:token for x, token in zip(X.flatten(), labels)}

In [52]:
df_pct_tok = df_pct.copy()
for col in df_pct:
    df_pct_tok[col] = df_pct[col].map(mapping)

In [53]:
df_pct_tok.head()

,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADI,ADM,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
2012-04-02,196,18,135,73,86,167,0,162,86,75,...,121,135,104,104,196,75,39,1,36,92
2012-04-03,131,136,26,51,4,84,92,149,154,32,...,64,183,119,64,9,88,40,38,161,104
2012-04-04,35,162,174,17,113,2,76,17,62,192,...,164,18,9,62,160,160,179,149,64,96
2012-04-05,147,159,76,1,53,88,88,76,199,127,...,193,163,63,38,191,36,167,95,63,117
2012-04-09,152,61,193,104,166,171,113,160,72,18,...,14,120,127,32,4,14,62,84,64,129


In [54]:
sentences = df_pct_tok.values.tolist()

In [55]:
model = gensim.models.Word2Vec(sentences, min_count=1)

In [56]:
embeddings = pd.DataFrame()
for col in df_pct_tok:
    embedding = np.zeros((100,))
    for i in df_pct_tok[col]:
        embedding += model.wv.get_vector(i)
    embedding /= df_pct_tok.shape[0]
    embeddings = pd.concat((embeddings, pd.Series(embedding)), axis=1)

In [57]:
embeddings

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0.027565,0.084233,0.014973,0.027445,-0.011365,0.069233,-0.001709,-0.001617,0.013993,0.017029,...,0.018692,0.067669,-0.014854,-0.007957,0.000877,0.016008,0.011564,0.001474,0.031436,0.037232
1,0.032141,0.090952,0.016714,0.028802,0.012772,0.082311,0.010313,0.010974,0.021461,0.031093,...,0.024663,0.038703,0.007866,-0.008360,0.007297,0.023629,0.020552,0.015740,0.041307,0.056158
2,-0.113984,-0.134170,-0.109942,-0.115079,-0.094903,-0.126197,-0.101154,-0.101488,-0.108481,-0.109168,...,-0.110977,-0.131435,-0.096284,-0.100731,-0.104433,-0.109617,-0.107385,-0.102957,-0.113574,-0.115417
3,0.031708,-0.113313,0.066647,0.035482,0.109167,-0.081654,0.100620,0.097602,0.062638,0.048521,...,0.052019,-0.039281,0.121535,0.127470,0.098966,0.057521,0.067348,0.088661,0.015696,-0.009640
4,-0.008769,0.159147,-0.047586,-0.009883,-0.124400,0.114559,-0.098368,-0.097481,-0.048238,-0.038456,...,-0.034443,0.111025,-0.135156,-0.117935,-0.090256,-0.043577,-0.054912,-0.088452,0.002011,0.025617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.100569,0.113437,0.096698,0.100198,0.091225,0.109727,0.091986,0.093340,0.095821,0.097327,...,0.097520,0.105693,0.089911,0.089065,0.092855,0.097166,0.095844,0.093239,0.100680,0.103216
96,0.050965,0.063677,0.048493,0.050375,0.053507,0.064546,0.049894,0.049951,0.051743,0.055065,...,0.050698,0.046646,0.050918,0.044108,0.046786,0.051179,0.050617,0.051725,0.056064,0.060135
97,-0.018459,-0.073064,-0.005612,-0.018749,0.024780,-0.056997,0.012573,0.012153,-0.003715,-0.005343,...,-0.008937,-0.062743,0.027295,0.016712,0.008827,-0.005623,-0.001805,0.010002,-0.020384,-0.024860
98,-0.137874,-0.048097,-0.159432,-0.140530,-0.178695,-0.063997,-0.176904,-0.175790,-0.155171,-0.144157,...,-0.150049,-0.102599,-0.187061,-0.196785,-0.178229,-0.152435,-0.157686,-0.169551,-0.125779,-0.107933


In [58]:
embeddings.T.to_csv("signal2vec_embds2012.csv", index=None)

In [14]:
a = pd.read_csv('data/embeddings/usa_2018/signal2vec_embds.csv')
a.index = df_pct.columns

In [15]:
a.to_csv("data/embeddings/usa_2018/signal2vec_embds.csv")